In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [40]:
path = 'shakespeare.txt'

In [41]:
text = open(path, 'r').read()

In [42]:
print(text[:1000])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.


                     2
  When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tattered weed of small worth held:  
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy lusty days;
  To say within thine own deep su

In [43]:
vocab = sorted(set(text))
print(vocab)
len(vocab)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '}']


84

In [44]:
for pair in enumerate(vocab):
    print(pair)

(0, '\n')
(1, ' ')
(2, '!')
(3, '"')
(4, '&')
(5, "'")
(6, '(')
(7, ')')
(8, ',')
(9, '-')
(10, '.')
(11, '0')
(12, '1')
(13, '2')
(14, '3')
(15, '4')
(16, '5')
(17, '6')
(18, '7')
(19, '8')
(20, '9')
(21, ':')
(22, ';')
(23, '<')
(24, '>')
(25, '?')
(26, 'A')
(27, 'B')
(28, 'C')
(29, 'D')
(30, 'E')
(31, 'F')
(32, 'G')
(33, 'H')
(34, 'I')
(35, 'J')
(36, 'K')
(37, 'L')
(38, 'M')
(39, 'N')
(40, 'O')
(41, 'P')
(42, 'Q')
(43, 'R')
(44, 'S')
(45, 'T')
(46, 'U')
(47, 'V')
(48, 'W')
(49, 'X')
(50, 'Y')
(51, 'Z')
(52, '[')
(53, ']')
(54, '_')
(55, '`')
(56, 'a')
(57, 'b')
(58, 'c')
(59, 'd')
(60, 'e')
(61, 'f')
(62, 'g')
(63, 'h')
(64, 'i')
(65, 'j')
(66, 'k')
(67, 'l')
(68, 'm')
(69, 'n')
(70, 'o')
(71, 'p')
(72, 'q')
(73, 'r')
(74, 's')
(75, 't')
(76, 'u')
(77, 'v')
(78, 'w')
(79, 'x')
(80, 'y')
(81, 'z')
(82, '|')
(83, '}')


In [45]:
char_to_ind = {u:i for i, u in enumerate(vocab)}
ind_to_char = np.array(vocab)

In [46]:
print(text[1000:1500])

nken eyes,
  Were an all-eating shame, and thriftless praise.
  How much more praise deserved thy beauty's use,
  If thou couldst answer 'This fair child of mine
  Shall sum my count, and make my old excuse'
  Proving his beauty by succession thine.
    This were to be new made when thou art old,
    And see thy blood warm when thou feel'st it cold.


                     3
  Look in thy glass and tell the face thou viewest,
  Now is the time that face should form another,
  Whose fresh repair i


In [47]:
random_phrase = '''Were an all-eating shame, and thriftless praise.
  How much more praise deserved thy beauty's use,
  If thou couldst answer 'This fair child of mine
  Shall sum my count, and make my old excuse'''

In [48]:
len(random_phrase)

193

In [49]:
random_line = "Feed'st thy light's flame with self-substantial fuel,"

In [50]:
len(random_line)

53

In [51]:
seq_len = 200

In [52]:
num_seq = len(text) // (seq_len + 1)

In [53]:
num_seq

27092

In [54]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [55]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [56]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

In [57]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt
#on avance par une time step

In [58]:
dataset = sequences.map(create_seq_targets)

In [59]:
dataset

<MapDataset element_spec=(TensorSpec(shape=(200,), dtype=tf.int32, name=None), TensorSpec(shape=(200,), dtype=tf.int32, name=None))>

In [60]:
batch_size = 256
buffer_size = 10000

In [61]:
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
#So model doesn't work on a particular ordering of the text (we shuffle batches)

In [62]:
vocab_size = len(vocab)
embed_dim = 64
rnn_neurons = 1026

In [63]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU

In [64]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [65]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [66]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [67]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

In [68]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (256, None, 64)           5376      
                                                                 
 gru_1 (GRU)                 (256, None, 1026)         3361176   
                                                                 
 dense_1 (Dense)             (256, None, 84)           86268     
                                                                 
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [69]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions)


tf.Tensor(
[[[-3.71791888e-03  7.01451208e-04  4.73481789e-03 ... -7.13015266e-04
    6.78933226e-04 -8.99071712e-03]
  [-5.03376126e-03  1.48599269e-03  4.40644007e-03 ...  6.28586998e-03
   -5.34234988e-03  2.04203115e-03]
  [ 2.92305974e-03  2.97383848e-03 -1.63177564e-03 ... -5.02615527e-04
   -2.85967160e-03 -2.92216497e-03]
  ...
  [ 2.27786042e-03 -5.08434745e-03  3.13166366e-03 ...  2.55458639e-03
   -5.21263340e-03  7.42846122e-03]
  [ 5.82807057e-04 -1.72818045e-03  7.05441274e-03 ...  1.47271820e-03
   -4.51057265e-03  9.21404362e-03]
  [-3.41917505e-04  1.31886359e-03  5.68184955e-03 ... -4.39737272e-03
   -1.19768200e-03  8.64605326e-03]]

 [[-5.85593225e-05  2.27798102e-03  1.96681777e-03 ... -2.12591700e-03
    2.63617164e-03 -2.44808523e-03]
  [ 4.34528702e-05 -8.73637677e-04 -3.39408475e-03 ... -1.20290299e-03
    5.54935867e-03 -1.61473011e-03]
  [-2.25443975e-03 -1.77946480e-04  1.50021719e-04 ...  5.70693798e-03
   -2.30755191e-03  4.86269686e-03]
  ...
  [ 4.011798

In [70]:
example_batch_predictions

<tf.Tensor: shape=(256, 200, 84), dtype=float32, numpy=
array([[[-3.71791888e-03,  7.01451208e-04,  4.73481789e-03, ...,
         -7.13015266e-04,  6.78933226e-04, -8.99071712e-03],
        [-5.03376126e-03,  1.48599269e-03,  4.40644007e-03, ...,
          6.28586998e-03, -5.34234988e-03,  2.04203115e-03],
        [ 2.92305974e-03,  2.97383848e-03, -1.63177564e-03, ...,
         -5.02615527e-04, -2.85967160e-03, -2.92216497e-03],
        ...,
        [ 2.27786042e-03, -5.08434745e-03,  3.13166366e-03, ...,
          2.55458639e-03, -5.21263340e-03,  7.42846122e-03],
        [ 5.82807057e-04, -1.72818045e-03,  7.05441274e-03, ...,
          1.47271820e-03, -4.51057265e-03,  9.21404362e-03],
        [-3.41917505e-04,  1.31886359e-03,  5.68184955e-03, ...,
         -4.39737272e-03, -1.19768200e-03,  8.64605326e-03]],

       [[-5.85593225e-05,  2.27798102e-03,  1.96681777e-03, ...,
         -2.12591700e-03,  2.63617164e-03, -2.44808523e-03],
        [ 4.34528702e-05, -8.73637677e-04, -3.3

In [71]:
model.fit(dataset,epochs=5)

Epoch 1/5
105/105 [==============================] - 941s 9s/step - loss: 3.3119
Epoch 2/5
105/105 [==============================] - 930s 9s/step - loss: 2.3946
Epoch 3/5
105/105 [==============================] - 937s 9s/step - loss: 2.0971
Epoch 4/5
105/105 [==============================] - 932s 9s/step - loss: 1.9060
Epoch 5/5
105/105 [==============================] - 933s 9s/step - loss: 1.7449


In [93]:
model.build(tf.TensorShape([1, None]))

In [92]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
    num_generate = gen_size
    input_eval = [char_to_ind[s] for s in start_seed]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = temp
    model.reset_states()
  
    for i in range(num_generate):

        predictions = model.predict(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(ind_to_char[predicted_id])
    return (start_seed + ''.join(text_generated))

In [94]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (256, None, 64)           5376      
                                                                 
 gru_1 (GRU)                 (256, None, 1026)         3361176   
                                                                 
 dense_1 (Dense)             (256, None, 84)           86268     
                                                                 
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [95]:
model2 = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

In [97]:
from tensorflow.keras.models import load_model

In [98]:
model.save('shakespearNLP.h5')

In [99]:
model2.load_weights('shakespearNLP.h5')

In [100]:
model2.build(tf.TensorShape([1, None]))


In [101]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (1, None, 64)             5376      
                                                                 
 gru_2 (GRU)                 (1, None, 1026)           3361176   
                                                                 
 dense_2 (Dense)             (1, None, 84)             86268     
                                                                 
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [102]:
print(generate_text(model2,"flower",gen_size=1000))

1/1 [==============================] - 0s 38ms/step
flower,
    In you ruve the be theem liee,
    Sinsce, here. Mait the tlostour, my and my faster sings,
    Yor for as eight hos. aw had I'll sen
    Th' anch'dows he dechndmer; and lasers avor endstre sabe.
    You houst not a cuntocoor as that?
  LACSAFAN. So it Warl; to charing; noblifl this saikiniod!
                 Axit SPMANICERS] IGow! TArisit Yebrobes,
    And tains to too.       REdVARK. His difore jedting with they?
  RINHTIO. I lay, mund with the; in besby gheol lige the wenl;
    The alemeruss o'llf laty sref the guentlikss, low-
  Plyoubsale for our of then this
    havess prasunttisblot, and he sake age-the ind shepe,
    Bucater, make, Yee, be to, the goods dom.

           E tomenthen, ISw. Geavel here
    you that sherl Het to come. Foo the greamy
    procher him! Ut shampen dind you costing- there
    'lingless bing, I'll ge spart, Wit I thmmen orat lonks,
    When im she to you thinkity tongus deard, his will hatt